In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import sys

# Получаем абсолютный путь к корневой директории проекта (директория выше текущей)
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Добавляем корневую директорию в sys.path
if root_path not in sys.path:
    sys.path.append(root_path)

In [8]:
import torch
from utils import parse_yaml
from models.clap_encoder import CLAP_Encoder
import faiss

In [6]:
SS_CONFIG_PATH = '../config/audiosep_base.yaml'
CLAP_CKPT_PATH = '../checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt'

In [7]:
device = torch.device('cuda')
configs = parse_yaml(SS_CONFIG_PATH)

query_encoder = CLAP_Encoder(pretrained_path = CLAP_CKPT_PATH).eval().to(device)

/home/araxal/anaconda3/envs/AudioSep/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassifi

In [94]:
embeddings = query_encoder.get_query_embed(modality='text', text=['vocal', 'drums', 'guitar']).cpu()
queries = query_encoder.get_query_embed(modality='text', text=['kick', 'ukulele', 'singing']).cpu()
embeddings.shape

torch.Size([3, 512])

In [95]:
index = faiss.IndexHNSWFlat(512, 32)

In [96]:
index.add(embeddings)

In [97]:
D, I = index.search(queries, k = 2)

In [98]:
D

array([[1.0701144 , 1.3757949 ],
       [1.1907144 , 1.3707284 ],
       [0.50372696, 0.9429971 ]], dtype=float32)

In [99]:
I

array([[1, 0],
       [2, 0],
       [0, 1]])